In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


# Visualizations

In [4]:
import os
import pandas as pd

from virny.utils.custom_initializers import create_models_metrics_dct_from_database_df

from source.utils.db_functions import read_model_metric_dfs_from_db
from source.custom_classes.experiments_composer import ExperimentsComposer
from source.custom_classes.experiments_visualizer import ExperimentsVisualizer

## Initialize Configs

In [5]:
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
DATASET_NAME = 'Ricci'
EXPERIMENT_SESSION_UUID = '431019cf-ba79-4256-8478-509186d8b0b5'
SENSITIVE_ATTRS = ['Race']
MODEL_NAMES = ['RandomForestClassifier']

In [6]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

## Group Metrics Composition

In [7]:
model_metric_dfs = read_model_metric_dfs_from_db(collection_obj, EXPERIMENT_SESSION_UUID)
models_metrics_dct = create_models_metrics_dct_from_database_df(model_metric_dfs)
client.close()

In [8]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].shape

(209, 18)

In [9]:
models_metrics_dct[list(models_metrics_dct.keys())[0]].head(20)

,Metric,Bootstrap_Model_Seed,Model_Name,Model_Params,Run_Number,Dataset_Name,Num_Estimators,Tag,Record_Create_Date_Time,Session_Uuid,Experiment_Iteration,Dataset_Split_Seed,Model_Init_Seed,Fair_Intervention_Params_Lst,Intervention_Param,Race_dis,Race_priv,overall
0,Accuracy,101,RandomForestClassifier,"{'bootstrap': False, 'ccp_alpha': 0.0, 'class_...",Run_1,Ricci,200,OK,2023-08-06 14:48:31.757,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.2,1.000000,1.000000,1.000000
1,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:53:33.961,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.7,0.800000,1.000000,0.916667
2,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:54:34.570,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.8,0.800000,1.000000,0.916667
3,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:55:35.030,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.9,0.800000,1.000000,0.916667
4,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:56:35.676,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",1.0,0.800000,1.000000,0.916667
5,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:46:36.384,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.0,1.000000,1.000000,1.000000
6,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:47:36.771,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.1,1.000000,1.000000,1.000000
7,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:49:32.404,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.3,1.000000,1.000000,1.000000
8,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:50:32.798,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.4,1.000000,1.000000,1.000000
9,Accuracy,101,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_w...",Run_1,Ricci,200,OK,2023-08-06 14:51:33.026,431019cf-ba79-4256-8478-509186d8b0b5,Exp_iter_1,100,100,"[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, ...",0.5,1.000000,1.000000,1.000000


In [10]:
exp_composer = ExperimentsComposer(models_metrics_dct, SENSITIVE_ATTRS)
exp_subgroup_metrics_dct = exp_composer.create_exp_subgroup_metrics_dct()

In [11]:
exp_group_metrics_dct = exp_composer.compose_group_metrics(exp_subgroup_metrics_dct)

## Metrics Visualization and Reporting

In [12]:
visualizer = ExperimentsVisualizer(exp_subgroup_metrics_dct=exp_subgroup_metrics_dct,
                                   exp_group_metrics_dct=exp_group_metrics_dct,
                                   dataset_name=DATASET_NAME,
                                   model_names=MODEL_NAMES,
                                   sensitive_attrs=SENSITIVE_ATTRS)

In [13]:
visualizer.melted_all_group_metrics_per_model_dct[MODEL_NAMES[0]].head(20)

,Metric,Model_Name,Experiment_Iteration,Intervention_Param,Group,Metric_Value
0,Accuracy_Parity,RandomForestClassifier,Exp_iter_1,0.2,Race,0.000000
1,Disparate_Impact,RandomForestClassifier,Exp_iter_1,0.2,Race,1.000000
2,Equalized_Odds_FNR,RandomForestClassifier,Exp_iter_1,0.2,Race,0.000000
3,Equalized_Odds_FPR,RandomForestClassifier,Exp_iter_1,0.2,Race,0.000000
4,Equalized_Odds_TPR,RandomForestClassifier,Exp_iter_1,0.2,Race,0.000000
5,IQR_Parity,RandomForestClassifier,Exp_iter_1,0.2,Race,0.003071
6,Jitter_Parity,RandomForestClassifier,Exp_iter_1,0.2,Race,-0.001429
7,Label_Stability_Ratio,RandomForestClassifier,Exp_iter_1,0.2,Race,1.001431
8,Statistical_Parity_Difference,RandomForestClassifier,Exp_iter_1,0.2,Race,0.000000
9,Std_Parity,RandomForestClassifier,Exp_iter_1,0.2,Race,-0.006106


### Line band plots for subgroup metrics

In [14]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Accuracy', 'F1', 'FNR', 'Jitter', 'Label_Stability', 'FPR'])

alt.VConcatChart(...)

In [15]:
visualizer.create_subgroup_metrics_line_band_plot(model_name=MODEL_NAMES[0],
                                                  subgroup_metrics=['Aleatoric_Uncertainty', 'Overall_Uncertainty'])

alt.VConcatChart(...)

### Line band plots for group metrics

In [16]:
visualizer.create_group_metrics_line_band_plot(group='Race',
                                               model_name=MODEL_NAMES[0],
                                               group_metrics=['Equalized_Odds_FPR', 'Equalized_Odds_FNR', 'Label_Stability_Ratio',
                                                              'Jitter_Parity', 'Disparate_Impact'])

alt.VConcatChart(...)